<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# 4. Modelos de lenguajes (libreta de curso)

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



#### Julio Waissman Vilanova (julio.waissman@unison.mx)





En esta libreta vamos a repasar los conceptos para la creación y análisis de modelos probabilísticos de secuencias de palabras (o *tokens*). Una exceente explicación sobre este tema se puede encontrar en [este capítulo del libro SLP](https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf).

## 4.1 Modelos de lenguajes por $n$-gramas


Vamos a desarrollar paso a paso uno de los modelos probabilísticos de lenguaje: el modelo de trigramas, asumiendo la hipótesis que la probabilidad que en un texto aparezca una palabra, depende *únicamente* de las dos palabras anteriores. Esto és, la probabilidad de la existencia de un trigrama específico en una posición dada, conociendo el bigrama de las dos primeras palabras del trigrama. 

El modelo más simple se entrena básicamente contando trigramas y contando bigramas. Para esto vamos a utilizar dos estructuras de datos que provée *python*: `Counter` y `defaultdict`. `Counter` genera una estructura que hereda de `dic` y que permite contar el número de ocurrencias de cada elemento en una secuencia (o iterable, como se dice en el argot *pythonero*).

Vamos a encontrar y numerar todos los trigramas de la obra *Martín Fierro el Gaucho*, y para eso vamos a separar la obra en frases y cada frase en palabras (dos *tokenizadores*)

In [1]:
import re
from random import randint
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize

archivo = "datos/martin_fierro.txt"

with open(archivo, 'r', encoding='utf8') as fp:
    texto_crudo = fp.read()

# Definimos el tokenizador en frases
# ¿Que pasaría si usamos palabras en minúscula y mayuscula?
# ¿Y si no eliminamos los signos de puntuación?
def tokenizador_frases(texto):
    # Minúsculas
    texto_preparado = texto.lower()
    # Eliminamos números
    texto_preparado = re.sub(r'[0-9]+', '', texto_preparado)
    # Cambiamos 'Ud.' por 'usted'
    texto_preparado = re.sub(r'Ud\.', 'usted', texto_preparado)
    # Separamos el texto en sentencias
    frases = sent_tokenize(texto_preparado, language='spanish')
    return frases

# definimos el tokenizador de palabras
# tokenizador_palabras = lambda sentencia: word_tokenize(sentencia, language='spanish')
tokenizador_palabras = lambda sentencia: regexp_tokenize(sentencia, r'\w+')

#Ejemplo
i = randint(0, 300)
print("En la linea {} se encuentra la frase: \n\n{}".format(i, tokenizador_frases(texto_crudo)[i]))
print("\n\nSeparada en tokens es: {}".format(tokenizador_palabras(tokenizador_frases(texto_crudo)[i])))

En la linea 220 se encuentra la frase: 

si salen a perseguir
después de mucho aparato,
tuitos se pelan al rato
y va quedando el tendal:
esto es como en un nidal
echarle güevos a un gato.


Separada en tokens es: ['si', 'salen', 'a', 'perseguir', 'después', 'de', 'mucho', 'aparato', 'tuitos', 'se', 'pelan', 'al', 'rato', 'y', 'va', 'quedando', 'el', 'tendal', 'esto', 'es', 'como', 'en', 'un', 'nidal', 'echarle', 'güevos', 'a', 'un', 'gato']


Y ahora, vamos a definir un generador, el cual vamos a usar para calcular cuantas veces aparecen cada uno de los trigramas, utilizando la función `trigrams` del módulo *nltk*. 

In [2]:
from nltk import trigrams
from collections import Counter

tri_generador = (
    trigrama for frase in tokenizador_frases(texto_crudo)
        for trigrama in trigrams(
            tokenizador_palabras(frase), 
            pad_left = True, left_pad_symbol="<s>",
            pad_right = True, right_pad_symbol='</s>'
        )
)    
tri_cuentas = Counter(tri_generador)

trigramas = list(tri_cuentas.keys())
ejemplo = trigramas[randint(0, len(trigramas) - 1)]

print("Los trigramas que más aparecen son:")
for (trig, cuenta) in tri_cuentas.most_common(20):
    print("\t{:20}\t({} veces)".format(' '.join(trig), cuenta))      

Los trigramas que más aparecen son:
	<s> <s> y           	(67 veces)
	<s> <s> yo          	(26 veces)
	<s> <s> no          	(19 veces)
	<s> <s> si          	(15 veces)
	<s> <s> el          	(13 veces)
	<s> <s> a           	(12 veces)
	<s> <s> pero        	(12 veces)
	lo mesmo que        	(11 veces)
	<s> <s> me          	(11 veces)
	ahi no más          	(9 veces)
	<s> <s> en          	(9 veces)
	<s> <s> por         	(8 veces)
	<s> <s> que         	(8 veces)
	<s> <s> al          	(8 veces)
	<s> <s> cuando      	(8 veces)
	que el hombre       	(8 veces)
	<s> <s> se          	(7 veces)
	<s> <s> ah          	(7 veces)
	<s> <s> era         	(7 veces)
	<s> <s> de          	(7 veces)


Y para obtener un modelo, pues simplemente dividimos el número de veces que aparece cada trigrama, entre el número de veces que aparece el bigrama conformado con las dos primeras palabras del trigrama. 

In [5]:
from nltk import bigrams
from collections import defaultdict

bi_generador = (
    bigrama for frase in tokenizador_frases(texto_crudo) 
        for bigrama in bigrams(
            ["<s>"] + tokenizador_palabras(frase), 
            pad_left = True, left_pad_symbol="<s>",
            pad_right = True, right_pad_symbol='</s>'
        )
)    
bi_cuentas = Counter(bi_generador)

modelo = defaultdict(lambda: 0)  #  Si el trigrama no aparece la probabilidad es 0
for trigrama in trigramas:
    modelo[trigrama] = tri_cuentas[trigrama] / bi_cuentas[trigrama[:-1]]
    
print(modelo[('<s>', '<s>', 'y')])
print(modelo[('y', 'ansí', 'estaba')])

0.13480885311871227
0


3

Como vemos en este caso, si un trigrama no aparece *exactamente* en el texto, el modelo asume una probabilidad 0. Estos es muy peligroso, ya que, debido a la hipótesis de independencia condicional que hicimos, la probabilidad de una frase es la multiplicación de la probabilidad de sus trigramas. Este problema lo resolveremos más adelante.

Por el momento veamos que tipos de frases se generan a partir de nuestro modelo.

In [ ]:
from random import random

def generador_sentencia(modelo):
    trigramas = modelo.keys()
    cadena = ['<s>','<s>']
    while cadena[-1] != '</s>':
        w1_w2 = (cadena[-2], cadena[-1])
        rdn = random()
        for t in trigramas:
            if t[:-1] == w1_w2:
                rdn -= modelo[t]
            if rdn <= 0:
                cadena.append(t[-1])
                break
    return ' '.join(cadena[2:-1])

for i in range(5):
    print(generador_sentencia(modelo))
    print(30*'-')    

Debido a que el *corpus* es relativamente pequeño algunas frases generadas son exactamente iguales a las del texto original. Sin embargo la gran mayoría son diferentes. Si entrenamos con un corpus más grande (como las obras completas de Cervantes), las frases son claramente diferentes a las originales. 

Ahora veamos si podemos calcular la perplejidad de un documento respecto al modelo probabilístico. La perplejidad de un documento $d$ respecto al modelo de trigramas $m$ se calcula como:

\begin{align}
\mathcal{P}_m(d) &= p_m(d)^{1/N},\\ 
p_m(d) &= \prod_1^{N+1}p_m(w_i|w_{i-1}, w_{i-2})
\end{align}

donde $p_m(p_m(w_i|w_{i-1}, w_{i-2})$ es la probabilidad condicional del trigama $w_{i-2}, w_{i-1}, w_{i}$ que acabamos de calcular en el modelo y $N$ es el número de tokens del texto a evaluar. 

In [ ]:
def perplejidad(texto, modelo):
    # Generador
    tri_generador = (
        trigrama for frase in tokenizador_frases(texto)
            for trigrama in trigrams(
                tokenizador_palabras(frase), 
                pad_left = True, left_pad_symbol="<s>",
                pad_right = True, right_pad_symbol='</s>'
            )
    )
    N = 0
    p_sentencia = 1
    for trigrama in tri_generador:
        p_sentencia *= modelo[trigrama]
        N += 1
    if p_sentencia == 0:
        print("Cuidado! La probabilidad de la frase es 0 y la perplejidad infinita")
        return None
    return (1/p_sentencia)**(1/N)

frase_1 = "Y ya me había hallao más prendido que un botón."
frase_2 = "Y ya me había hallao más prendido que un baile."

print("La frase <<{}>> tiene una perplejidad de {}\n".format(frase_1, perplejidad(frase_1, modelo)))
print("La frase <<{}>> tiene una perplejidad de {}".format(frase_2, perplejidad(frase_2, modelo)))

Como podemos ver, inclusive en frases que utilizan el mismo vocabulario del texto, la perplejidad se vuelve infinita. Es por esa razón que es necesario *suavizar* el modelo, asumiendo que existe un grado de ignorancia estadística en nuestra evidencia (el *corpus*). El método más usado en PLN de suavizado es el llamado *Kneser Ney*, el cual es laborioso de implementar, una explicación a grandes razgos del método la puedes encontrar [aqui](http://www.foldl.me/2014/kneser-ney-smoothing/).

Afortunadamente, en *nltk* ya existe la distribución discreta con suavizado de Kneser Ney*. 

In [ ]:
from math import pow
from nltk import FreqDist, KneserNeyProbDist

def modelado_kn(corpus, tokenizador_palabras, tokenizador_frases):
    return KneserNeyProbDist(
        FreqDist(generador_trigramas(corpus, tokenizador_palabras, tokenizador_frases))
    )

def generador_trigramas(texto, tokenizador_palabras, tokenizador_frases):
    tri_generador = (
        trigrama for frase in tokenizador_frases(texto) 
             for trigrama in trigrams(
                tokenizador_palabras(frase), 
                pad_left = True, left_pad_symbol="<s>",
                pad_right = True, right_pad_symbol='</s>'
        )
    ) 
    return tri_generador        

def genera_sentencia(modelo):
    trigramas = list(modelo.samples())
    cadena = ['<s>','<s>']
    while cadena[-1] != '</s>':
        w1_w2 = (cadena[-2], cadena[-1])
        rdn = random()
        for trigrama in trigramas:
            if trigrama[:-1] == w1_w2:
                rdn -= modelo.prob(trigrama)
            if rdn <= 0:
                cadena.append(trigrama[-1])
                break
    return ' '.join(cadena[2:-1])

def perplejidad(texto, modelo):
    vocab = {t[2] for t in modelo.samples()}
    N = 0
    p = 1
    for trigrama in generador_trigramas(texto, tokenizador_palabras, tokenizador_frases):
        if trigrama[-1] not in vocab:
            print("Cuidado! La probabilidad de la frase es 0 y la perplejidad infinita")
            return None            
        lista_trigramas = [t for t in modelo.samples() if t[:-1] == trigrama[:-1]]
        if not lista_trigramas:
            p *= 1/len(vocab)
        if trigrama in lista_trigramas:
            p *= modelo.prob(trigrama)
        else:
            no_p = sum(modelo.prob(t) for t in lista_trigramas)
            p *= (1 - no_p)
        N += 1
    return (1 / p) ** (1 / N)

Y ahora vamos a probarlo

In [ ]:
modelo_kn = modelado_kn(texto_crudo, tokenizador_palabras, tokenizador_frases)

for _ in range(5):
    print(genera_sentencia(modelo_kn))
    print(30 * '-')
 
frase_1 = "Y ya me había hallao más prendido que un botón."
frase_2 = "Y ya me había hallao más prendido que un baile."
frase_3 = 'Y ya me habia hallao más cosida que un botón.'
frase_4 = "Y aquí estoy poniendo una frase que seguro no es del libro para saber su perplejidad."

p_str = "La frase <<{}>> tiene una perplejidad de {}\n"
print(p_str.format(frase_1, perplejidad(frase_1, modelo_kn)))
print(p_str.format(frase_2, perplejidad(frase_2, modelo_kn)))
print(p_str.format(frase_3, perplejidad(frase_3, modelo_kn)))
print(p_str.format(frase_4, perplejidad(frase_4, modelo_kn)))

Sin embargo, en este modelo, si no encuentra un bigrama existente, tambien calcula una perplejidad infinita a una frase que no estaría muy lejana. 

Para resolver esto, es necesario utilizar el concepto de *palabras fuera de vocabulario (OOV)* y esto se logra con un diccionario preestablecido, o generando nuevos tokens en el texto. Vamos a generar un nuebo modelo que utilice palabras OOV (marcadas como '<UNK>'). Vamos a construir el vocabulario, y cada vez que se encuentre por primera vez una palabra la vamos a substituir por <UNK>. Si aparece una segunda vez, ya queda en su forma original. 

In [ ]:
def modelado_knOOV(corpus, tokenizador_palabras, tokenizador_frases):
    vocabulario = set([])
    modelo = KneserNeyProbDist(
        FreqDist(generador_trigramasOOV(corpus, tokenizador_palabras, tokenizador_frases, vocabulario))
    )
    return modelo, vocabulario

def generador_trigramasOOV(texto, tokenizador_palabras, tokenizador_frases, vocab):
    vocab_ini = set([])
    if len(vocab) == 0:
        def token(frase):
            for t in tokenizador_palabras(frase):
                if t not in vocab_ini:
                    vocab_ini.add(t)
                    yield '<UNK>'
                else:
                    vocab.add(t)
                    yield t
    else:
        def token(frase):
            for t in tokenizador_palabras(frase):
                yield t if t in vocab else '<UNK>'
    tri_generador = (
        trigrama for frase in tokenizador_frases(texto) 
             for trigrama in trigrams(
                token(frase), 
                pad_left = True, left_pad_symbol="<s>",
                pad_right = True, right_pad_symbol='</s>'
        )
    ) 
    return tri_generador    

def perplejidadOOV(texto, modelo, vocab):
    N = 0
    p = 1
    for trigrama in generador_trigramasOOV(texto, tokenizador_palabras, tokenizador_frases, vocab):
        lista_trigramas = [t for t in modelo.samples() if t[:-1] == trigrama[:-1]]
        if not lista_trigramas:
            p *= 1/(len(vocab) + 1)
        if trigrama in lista_trigramas:
            p *= modelo.prob(trigrama)
        else:
            no_p = sum(modelo.prob(t) for t in lista_trigramas)
            p *= (1 - no_p)/(len(vocab) - len(lista_trigramas))
        N += 1
    return (1 / p) ** (1 / N)        

Y ahora lo aplicamos

In [ ]:
modelo_knOOV, vocab = modelado_knOOV(texto_crudo, tokenizador_palabras, tokenizador_frases)

for _ in range(5):
    print(genera_sentencia(modelo_knOOV))
    print(30 * '-')
    
frase_1 = "Y ya me había hallao más prendido que un botón."
frase_2 = "Y ya me había hallao más prendido que un baile."
frase_3 = 'Y ya me habia hallao más cosida que un botón.'
frase_4 = "Y aquí estoy poniendo una frase que seguro no es del libro para saber su perplejidad."

p_str = "La frase <<{}>> tiene una perplejidad de {}\n"
print(p_str.format(frase_1, perplejidadOOV(frase_1, modelo_knOOV, vocab)))
print(p_str.format(frase_2, perplejidadOOV(frase_2, modelo_knOOV, vocab)))
print(p_str.format(frase_3, perplejidadOOV(frase_3, modelo_knOOV, vocab)))
print(p_str.format(frase_4, perplejidadOOV(frase_4, modelo_knOOV, vocab)))

## 4.2. Marcado de *parte del discurso* (POST) y reconocimiento de entidades (NER)

El marcado de *parte del discurso (POST)* así como el *reconocimiento de entidades (NER)* son más o menos la misma tarea. Es interesante armar un modelo basado en *Campos aleatorios condicionales (CRF)*, pero el problema es que para su entrenamiento se requieren muchos ejemplos de texto anotado. El texto anotado es una de las cosas más difíciles de encontrar, por lo que, para estos problemas, se suelen usar modelos pre-entrenados.

En este apartado vamos a ver como utilizar *spacy* como herramienta tanto para POST como NER. Para esto, previamente hemos descargado el modelo en español de `spacy` de tamaño medio. Este modelo está entrenado bajo diferentes fuentes y en general funciona bastante bien. Para una explicación de como descargar el modelo (si lo quieres usar fuera del contenedor) y como fue entrenado, puedes consultar [aqui](https://spacy.io/models/es#section-es_core_news_md).

La biblioteca de *spacy* ha cobrado en el último año mucha fuerza, sobre todo en aplicaciones reale, ya que si bien tarda en cargar el modelo, es muy rápida para el procesamiento, y su operación es muy sencilla, como veremos.

In [ ]:
import pandas as pd
import spacy
nlp = spacy.load('es_core_news_sm')

Si bien, para el inglés, *spacy* mantiene [la notación POS propuesta por el proyecto *Penn Treebank*](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), en español utiliza su propia notación, que es bastante intuitiva y se puede consultar con el comando `spacy.explain`.

In [ ]:
tags = set([tag.strip('_').split('__')[0] for tag in spacy.lang.es.TAG_MAP.keys()])
print("{:8}{:15}".format("Tag", "Descripción"))
print(23*'-')
for tag in tags:
    print("{:8}{:15}".format(tag, spacy.explain(tag)))

Ahora si, vamos a generar el etiquetado POS para una frase.

In [ ]:
# Practica cambiando el texto, por otros textos (para la visualización, de preferencia cortos)
# texto = "En el 2018, elegí la universidad pública y gratuita."
# texto = "MapUNaM contiene información sobre actividades de extensión y de investigación que realiza la Universidad Nacional de Misiones en distintos puntos de la provincia."
texto = "Mi nombre es Julio Waissman Vilanova, trabajo en la Universidad de Sonora y quisiera conocer las Cataratas de Iguazú"
print('Texto a analizar: <<{}>>'.format(texto))

doc = nlp(texto)
tokens_dic = []
for token in doc:
    tokens_dic.append({
        'texto': token.text,
        'lema': token.lemma_,
        'Etiquetado POS simple': token.pos_,
        'Etiquetado POS completo' : token.tag_,
        '¿Palabra vacia?': token.is_stop,
        '¿Fuera de vocabulario?': token.is_oov,
        'Entidad': token.ent_type_,
        'Notación BIO': token.ent_iob_
    })
doc_dt = pd.DataFrame.from_dict(tokens_dic)

Y vamos a visualizar el etiquetado de la frase

In [ ]:
display(doc_dt[['texto', 'Etiquetado POS simple', 'Etiquetado POS completo', 
                '¿Palabra vacia?', '¿Fuera de vocabulario?']])

Como vemos, además del etiquetado POS sencillo, se hace un etiquetado completo que nos indica persona, genero, tiempo, plural o singular, etc. Igualmente, se nos avisa si esta palabra está considerada en el modelo como palabra vacía, o si no se encuentra registrada.

Si ahora utilizamos la misma frase ya procesada para el reconocimiento de entidades tenemos lo siguiente:

In [ ]:
display(doc_dt[['texto', 'Entidad', 'Notación BIO']])

for entidad in doc.ents:
    print("La etiqueta {} significa <<{}>>".format(entidad.label_, spacy.explain(entidad.label_)))

Es interesante que se mantiene la notación *BIO*. Como este análisis por palabras puede ser de utilidad para entender la estructura de una oración, *spacy* viene con un método de visualización de POST y de NER bastante atractivos. 

In [ ]:
from IPython.core.display import HTML

dependencias = spacy.displacy.render(doc, style='dep')
display(HTML(dependencias))

In [ ]:
entidades = spacy.displacy.render(doc, style='ent')
display(HTML(entidades))

Si no hay etiquetado NER en una oración, a adevolver un error.